In [198]:
%load_ext autoreload
%autoreload 2

%load_ext rpy2.ipython

d:\it\coding\github projects\my-university\iii курс\(python) эконометрика\venv\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning:

The symbol 'quartz' is not in this R namespace/package.



In [242]:
import pandas as pd
from src.stats import LinearRegression, add_constant
from src.report import general, general_assessment, f_test, t_test, conf_int, other
import statsmodels.api as sm
import plotly.express as pe

In [297]:
df = pd.read_csv('data/GAM_pure.csv', sep=',')

# R

# Python

In [298]:
y = df['y']
x = df[['x1', 'x2', 'x3', 'x4']]

In [299]:
model = LinearRegression(y, x)

In [300]:
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                              2846.
Date:                Sat, 09 Oct 2021   Prob (F-statistic):                    6.08e-25
Time:                        15:16:20   Log-Likelihood:                         -170.31
No. Observations:                  22   AIC:                                      348.6
Df Residuals:                      18   BIC:                                      353.0
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

# Автокорреляция

In [301]:
resid = model.results.resid
xs_resid_df = pd.concat([model.x, resid.rename('e')], axis=1)

## График

In [302]:
pe.scatter(x=resid.shift(-1), y=resid)

## Тесты

In [303]:
%Rpush df

In [329]:
%%R

library(lmtest)

m = lm(y ~ x1 + x2 + x3 + x4, data=df)
summary(m)


Call:
lm(formula = y ~ x1 + x2 + x3 + x4, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-953.50 -333.93   -9.69  187.71 1292.08 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)   
(Intercept) 10399.9333  6169.6976   1.686  0.11013   
x1              0.4838     0.1964   2.463  0.02473 * 
x2            130.3151   132.1259   0.986  0.33782   
x3             -6.4678    25.6981  -0.252  0.80430   
x4           -276.3727    78.9464  -3.501  0.00274 **
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 586.3 on 17 degrees of freedom
Multiple R-squared:  0.8064,	Adjusted R-squared:  0.7608 
F-statistic:  17.7 on 4 and 17 DF,  p-value: 6.833e-06



In [330]:
%%R

dw_pvalue = dwtest(m)[4]

In [331]:
%Rpull dw_pvalue

In [332]:
dw_pvalue = dw_pvalue[0][0]
dw_pvalue

0.0018184940229457095

In [288]:
dw = sm.stats.durbin_watson(resid)
dw

1.4553883532344523

In [287]:
bg_1_pvalue = sm.stats.acorr_breusch_godfrey(model.results, nlags=1)
bg_1_pvalue

(4.168015687441099,
 0.041194018326494795,
 0.9139479717455136,
 0.35329176480848595)

In [278]:
bg_2_pvalue = sm.stats.acorr_breusch_godfrey(model.results, nlags=2)[1]
bg_2_pvalue

0.1164186835709424

In [279]:
bg_3_pvalue = sm.stats.acorr_breusch_godfrey(model.results, nlags=3)[1]
bg_3_pvalue

0.05119153782466862

In [314]:
bg_4_pvalue = sm.stats.acorr_breusch_godfrey(model.results, nlags=4)[1]
bg_4_pvalue

0.056778162286703085

# Гетероскедастичность/Гомоскедастичность

## Графики

In [315]:
x1_sorted = xs_resid_df.sort_values(['x1'])
pe.scatter(x=x1_sorted['x1'], y=(x1_sorted['e'] ** 2), labels={'x': 'x1', 'y': 'e_1^2'})

In [316]:
x2_sorted = xs_resid_df.sort_values(['x2'])
pe.scatter(x=x2_sorted['x2'], y=(x2_sorted['e'] ** 2), labels={'x': 'x2', 'y': 'e_2^2'})

In [317]:
x3_sorted = xs_resid_df.sort_values(['x3'])
pe.scatter(x=x3_sorted['x3'], y=(x3_sorted['e'] ** 2), labels={'x': 'x3', 'y': 'e_3^2'})

In [318]:
x4_sorted = xs_resid_df.sort_values(['x4'])
pe.scatter(x=x4_sorted['x4'], y=(x4_sorted['e'] ** 2), labels={'x': 'x4', 'y': 'e_4^2'})

## Тесты

In [219]:
exog = model.results.model.exog

In [310]:
gq_pvalue = sm.stats.het_goldfeldquandt(resid, exog)[1]
gq_pvalue

0.01443013600496952

In [311]:
bp_pvalue = sm.stats.het_breuschpagan(resid, exog)[1]
bp_pvalue

0.039193864462519996